In [ ]:
from cv2 import VideoCapture, CAP_PROP_FPS, CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT, VideoWriter_fourcc, VideoWriter
from pandas import read_csv, DataFrame
from labels import get_labels_as_dataframe
from common import get_filename

In [2]:
from labels import get_label
get_label("../data/labels/How to Flag - A Climbing Technique for Achieving Balance.csv", 1500)

<Technique.INVALID: 0>

In [ ]:
labels = get_labels_as_dataframe("../data/labels/How to Flag - A Climbing Technique for Achieving Balance.csv")
labels

In [ ]:
labels.loc[0]

In [ ]:
def generate_samples(video_path):
    original_video = VideoCapture(video_path)
    if not original_video.isOpened():
        print(f"Cannot open video file '{video_path}'")
        exit()

    file_name = get_filename(video_path)
    fps = original_video.get(CAP_PROP_FPS)
    frame_width = original_video.get(CAP_PROP_FRAME_WIDTH)
    frame_height = original_video.get(CAP_PROP_FRAME_HEIGHT)
    frame_size = (int(frame_width),int(frame_height))

    label_path = video_path.replace("/videos/", "/labels/").replace(".mp4", ".csv")
    labels = get_labels_as_dataframe(label_path)

    frame_num = 0
    row_num = 0
    write_frames = False

    while (original_video.isOpened()):
        success, image = original_video.read()
        if not success or image is None:
            print(f'Could not read frame nr {frame_num}')
            break
            
        if frame_num == labels.loc[row_num, "frame"]:
            label_name = Technique(labels.loc[row_num, "label"]).name
            sample = VideoWriter(f"../data/samples/{label_name}/{file_name}__{frame_num}.mp4", VideoWriter_fourcc('d', 'i', 'v', 'x'), fps, frame_size)
            write_frames = True
        
        if write_frames:
            sample.write(image)
            print(frame_num)
        
        if frame_num == labels.loc[row_num+1, "frame"]:
            sample.release()
            
        frame_num += 1